# HK Variance Test

### This notebook provides a test for the noise and stability of the housekeeping data captured by the FPE.  It evaluates the performance of the housekeeping system by taking a large number of samples, collecting the results for each signal channel, and evaluating the variance of the data for each channel.

### Instructions:
Enter Your Name and today's date:

Ed Bokhour, 11/4/15

Enter the part numbers and serial numbers of the units under test:

SDPCB Interface 6.1, s/n 02. SDPCB 6.1 Driver s/n RPI15310002, SDPCB Video 6.1 s/n RPI15310001. Using wrapper 6.1.2 (San Diego).

When the test is complete, save this notebook as a new file, indicating the date, as "HK_Variance_Results_YYMMDD.ipynb".  Alternatively, export the notebook as a PDF file, then clear all entries and outputs.

### Start the Observatory Simulator and Load the FPE FPGA

Remember that whenever you power-cycle the Observatory Simulator, you should set `preload=True` below.

When you are running this notebook and it has *not* been power cycled, you should set `preload=False`.

Run the following cell to get the FPE loaded:

In [9]:
from tessfpe.dhu.fpe import FPE
from tessfpe.dhu.unit_tests import check_house_keeping_voltages
fpe1 = FPE(1, debug=False, preload=True, FPE_Wrapper_version='6.1.1')
print fpe1.version
fpe1.cmd_start_frames()
fpe1.cmd_stop_frames()
if check_house_keeping_voltages(fpe1):
    print "Wrapper load complete. Interface voltages OK."

Observatory Simulator Version 1.6c - Oct  8 2015 11:55:03
Wrapper load complete. Interface voltages OK.


### Set all the operating parameters to the default values:

In [10]:
def set_fpe_defaults(fpe):
    "Set the FPE to the default operating parameters and return a list of the default values"
    defaults = {}
    for k in range(len(fpe.ops.address)):
        if fpe.ops.address[k] is None:
            continue
        fpe.ops.address[k].value = fpe.ops.address[k].default
        defaults[fpe.ops.address[k].name] = fpe.ops.address[k].default
    return defaults
set_fpe_defaults(fpe1)

{'ccd1_backside': 0.0,
 'ccd1_input_diode_high': 12.0,
 'ccd1_input_diode_low': 12.0,
 'ccd1_input_gate_1': -3.0,
 'ccd1_input_gate_2': -3.0,
 'ccd1_output_drain_a_offset': 8.0,
 'ccd1_output_drain_b_offset': 8.0,
 'ccd1_output_drain_c_offset': 8.0,
 'ccd1_output_drain_d_offset': 8.0,
 'ccd1_output_gate': -1.0,
 'ccd1_parallel_high_offset': 0.0,
 'ccd1_parallel_low': -8.0,
 'ccd1_reset_drain': 12.0,
 'ccd1_reset_high': 3.0,
 'ccd1_reset_low_offset': -3.0,
 'ccd1_scupper': 12.0,
 'ccd1_serial_high': 4.0,
 'ccd1_serial_low_offset': -6.0,
 'ccd1_substrate': -40.0,
 'ccd2_backside': 0.0,
 'ccd2_input_diode_high': 12.0,
 'ccd2_input_diode_low': 12.0,
 'ccd2_input_gate_1': -3.0,
 'ccd2_input_gate_2': -3.0,
 'ccd2_output_drain_a_offset': 8.0,
 'ccd2_output_drain_b_offset': 8.0,
 'ccd2_output_drain_c_offset': 8.0,
 'ccd2_output_drain_d_offset': 8.0,
 'ccd2_output_gate': -1.0,
 'ccd2_parallel_high_offset': 0.0,
 'ccd2_parallel_low': -8.0,
 'ccd2_reset_drain': 12.0,
 'ccd2_reset_high': 3.0,
 'cc

### Run the variance test:

In [4]:
from numpy import var, sqrt
samples=100
from tessfpe.data.housekeeping_channels import housekeeping_channels
# We make sample_data a dictionary and each value will be a set of HK 
# data, with key = sample_name.
sample_data = {}

# For later:
signal_names = []
signal_values = []
signal_data = {}
variance_values = {}

for i in range(samples):
    # Get a new set of HK values
    house_keeping_values = fpe1.house_keeping["analogue"]
    data_values = house_keeping_values.values()
    # Add the new HK values to the sample_data dictionary:
    sample_number = "sample_" + str(i)
    sample_data[sample_number] = data_values

# Get the signal names for use later
signal_names = housekeeping_channels.keys()

# Get list of units for later
units = {}
for name in housekeeping_channels:
    units[name] = housekeeping_channels[name]['unit']
    
"""Assign the set of all HK values of the same signal (e.g. substrate_1) 
to the dictionary 'signal_data'"""

for k in range(len(signal_names)):
    # Build the list 'signal_values' for this signal:
    for i in range(samples):
        sample_number = "sample_" + str(i)
        signal_values.append(sample_data[sample_number][k])
    # Add signal_values to the signal_data dictionary:
    signal_data[signal_names[k]] = signal_values
    signal_values = []

""" Now get the square root of the variance of each of the 'signal_values' in the 
signal_data dictionary and put the result in the 'variance_values' 
dictionary."""
for name in signal_data:
    variance_values[name] = sqrt(var(signal_data[name]))
    #print units[name]["unit"]
    #print signal_data
    #print units[name]
    print '{0:26}     {1:.3} {2}'.format(name, variance_values[name], units[name])

# Results will be displayed below, in engineering units (root-variance).  
# Watch ObsSim LEDs for activity.

ccd4_substrate                 0.0057 V
ccd1_serial_low_offset         0.00111 V
ccd2_output_source_a           0.00086 V
ccd2_output_source_c           0.000881 V
ccd2_output_source_b           0.000966 V
ccd2_output_source_d           0.000657 V
ccd4_parallel_low              0.000846 V
ccd4_output_drain_b_offset     0.00125 V
ccd1_output_source_d           0.000928 V
ccd3_output_source_a           0.00118 V
ccd3_output_source_b           0.00123 V
ccd3_output_source_c           0.000916 V
ccd3_output_source_d           0.000854 V
ccd4_output_source_d           0.00089 V
ccd4_output_source_c           0.0101 V
ccd4_output_source_b           0.000879 V
ccd4_output_source_a           0.000754 V
ccd3_board_temperature         0.000885 K
ccd2_output_drain_d_offset     0.000799 V
ccd1_output_drain_b_offset     0.0151 V
ccd2_output_gate               0.00103 V
ccd4_input_diode_low           0.000864 V
ccd3_substrate                 0.000696 V
ccd2_serial_low_offset         0.000694 V
ccd1_

## End HK Variance Test.